## Conexão e extração dos dados do Twitter

In [ ]:
!pip install textblob

In [ ]:
!pip install tweepy

In [1]:
import tweepy
import numpy as np

from nltk import word_tokenize
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict

In [2]:
# Variaveis de acesso, keys cedidas pela api do twitter
consumer_key = "D7WbZKB1D8sI6SG5YSJaoUqFw"
consumer_secret = "Ce2wShKqGIeMgHrmugmpNRK1AAGNXik1imJ47sLJnF4dhFKLqT"
access_token = "758066017742512128-UaPf9IJfaJ957JO5nb4F8TNVa7PBH3p"
access_token_secret = "amPLoPVU6hyOXUyNVRRVrzZPU6zP0lT8Mnt7eLxhbFzdF"

In [3]:
# Realizando a autenticação
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [4]:
# Realizando a conexão com a API
api = tweepy.API(auth)

In [5]:
# Variável que irá armazenar todos os Tweets com a palavra escolhida na função search da API
#public_tweets = api.search('Xiaomi')
public_tweets = api.search('xiaomi mi9', lang='pt')

In [6]:
#print(public_tweets)

In [7]:
#type(public_tweets)

## Manipulação dos dados para gerar um data frame com pandas

In [8]:
# Imprimir todos os tweets
# Cria a lista com algumas informações dos tweets extraidos

def cria_lista_tweets(public_tweets):
    tweets = [] # Lista vazia para armazenar scores
    for tweet in public_tweets:
        #print(tweet)
        print(tweet.id)
        print(tweet.created_at)
        print(tweet.text)
        print()

        lista_aux = []
        lista_aux.append(tweet.id)
        lista_aux.append(tweet.created_at)
        lista_aux.append(tweet.text)

        tweets.append(lista_aux)

    #print(tweets)
    return tweets

In [9]:
tweets = cria_lista_tweets(public_tweets)

1171920866181025792
2019-09-11 22:58:02
RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPhone 8, o Xiaomi leva a melhor e ainda é mais barato cerca de 120€. Parece-me incrível que a A…

1171909987976785922
2019-09-11 22:14:48
RT @compiws: serio que tem gente falando que xiaomi vai copiar as 3 cameras''fogão de 3 bocas'' do iphone 11,vei xiaomi fez isso antes que…

1171894182606245889
2019-09-11 21:12:00
RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPhone 8, o Xiaomi leva a melhor e ainda é mais barato cerca de 120€. Parece-me incrível que a A…

1171886123435679745
2019-09-11 20:39:59
Meu amigo -Mano eu já conhecia o xaomi, mas sempre tive receio de comprar, dei 4k no IPhone X.
Hoje chegou o Xiaomi… https://t.co/9uMtQyjiuX

1171876391819927553
2019-09-11 20:01:18
RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPhone 8, o Xiaomi leva a melhor e ainda é mais barato cerca de 120€. Parece-me incrível que a A…

1171875785621299202
2019-09-11 19:58:54
RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPhone 8

In [23]:
def cria_arquivo_csv(lista):
    arquivo = open('tweets_xiaomi.csv', 'w', encoding='utf-8')
  
    arquivo.write('id,created_at,text' + '\n')
    
    for tweet in lista:
        #print(tweet)
        for pos in range(len(tweet)):
            if pos == len(tweet)-1:
                arquivo.write('"' + str(tweet[pos]) + '"')
            else:
                arquivo.write(str(tweet[pos]))
                arquivo.write(',')
                
        arquivo.write('\n')
                
        
    arquivo.close()

In [24]:
cria_arquivo_csv(tweets)

In [25]:
dataset = pd.read_csv('tweets_xiaomi.csv', encoding='utf-8')

In [26]:
type(dataset)

pandas.core.frame.DataFrame

In [27]:
dataset.head(10)

,id,created_at,text
0,1171920866181025792,2019-09-11 22:58:02,RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPh...
1,1171909987976785922,2019-09-11 22:14:48,RT @compiws: serio que tem gente falando que x...
2,1171894182606245889,2019-09-11 21:12:00,RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPh...
3,1171886123435679745,2019-09-11 20:39:59,"Meu amigo -Mano eu já conhecia o xaomi, mas se..."
4,1171876391819927553,2019-09-11 20:01:18,RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPh...
5,1171875785621299202,2019-09-11 19:58:54,RT @cryptolobopt: Entre um Xiaomi Mi9 e um iPh...
6,1171844923265564672,2019-09-11 17:56:16,@BeyondTheWings_ Não vai ter eu ganhando um Xi...
7,1171840867197431810,2019-09-11 17:40:09,@dsouza_55 Faz essa merda não mano. 6s totalme...
8,1171838224718868480,2019-09-11 17:29:39,Quem quiser me da um xiaomi mi9 de presente to...
9,1171819246147506176,2019-09-11 16:14:14,"@kruda1020 Queria um Mi9 tbm, q Xiaomi é boa\r..."


In [29]:
# Contabiliza a quantidade de elementos do data frame
dataset.count()

id            15
created_at    15
text          15
dtype: int64

## Pre-processamento dos Dados

In [30]:
# Retorna o nome das colunas do data frame
dataset.columns

Index(['id', 'created_at', 'text'], dtype='object')

In [31]:
# Remove textos duplicados
dataset.drop_duplicates(['text'], inplace=True)

In [32]:
# Contabiliza a quantidade de elementos do data frame, depois de remover os dados duplicados
dataset.text.count()

10

In [33]:
import nltk
nltk.download('stopwords')
nltk.download('rslp')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\MaluF\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [34]:
def RemoveStopWords(instancia):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    palavras = [i for i in instancia.split() if not i in stopwords]
    return (" ".join(palavras))

In [35]:
def Stemming(instancia):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = []
    for w in instancia.split():
        palavras.append(stemmer.stem(w))
    return (" ".join(palavras))

In [36]:
def Limpeza_dados(instancia):
    # remove links, pontos, virgulas,ponto e virgulas dos tweets
    instancia = re.sub(r"http\S+", "", instancia).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    return (instancia)